In [1]:
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
from scapy.all import *
import glob
import numpy as np
# You can delete this later, I put, I dont like Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

In [3]:
#imported to match IP
import re

In [4]:
pcap =rdpcap("31_May.pcap")

In [5]:
# pcap[0].show()
# pcap[0].src
# pcap[0].dst
# pcap[0].time
# pcap[0].summary()

In [6]:
df= pd.DataFrame(columns=['Time',
                          'TP',
                          'Total_Bits',
                          'Total_Packets',
                          'Packet_Sizes',
                          'Packet_Times',
                          'IPGs'])
current_time=0; # Current Time
ipg=0; # IPG
window_size=1
first_pk=0
for i in np.arange(0, 350, window_size):
    ttl_bits=0 # Total Bits
    ttl_pk=0 # Total Packets
    pk_size=[] # Packet Size
    pk_time=[] # Packet Time
    ipg_array=[] # IPG Times
    
    for p in pcap:
        if ((p.haslayer(UDP)) and (p.haslayer(IP))) and ((float(p.time-pcap[0].time))>= i) and  ((float(p.time-pcap[0].time))< i+window_size):
            if(re.match('142', p.src)): #in other words it is source. if(p[IP].src== '142.*.*.*'): # Downlink
                #print(int(len(p[IP])))
                ttl_bits=ttl_bits+(int(len(p[IP]))*8) # This converts to bits
                ttl_pk=ttl_pk+1 # This total packets
                pk_size.append(int(len(p[IP]))) # This save sizes in an array
                pk_time.append(float(p.time)) # This save times in an array
                ipg=float(p.time-current_time)
                if(first_pk==0):
                    ipg_array.append(0)
                else:
                    ipg_array.append(ipg)
                current_time=p.time
                first_pk=first_pk+1;      
                # Here If Done
            #Outer If
    #For of packets
    TP=ttl_bits/window_size
    
    #print(ttl_pk)
    df = df.append(pd.Series([i,
                                  TP,
                                  ttl_bits,
                                  ttl_pk,
                                  pk_size,
                                  pk_time,
                                  ipg_array], index=df.columns ), ignore_index=True)
    df.to_csv ('31_qos.csv', index = None, header=True)            
        